# 🔗 Chain & Graph - 작업 체인 구성

이 노트북은 beanllm의 Chain과 Graph 기능을 학습합니다. 여러 작업을 순차적 또는 병렬로 연결하여 복잡한 워크플로우를 구성할 수 있습니다.

## 📋 목차

- [1. Chain 기본 사용](#1-chain-기본-사용)
- [2. Sequential Chain](#2-sequential-chain)
- [3. Parallel Chain](#3-parallel-chain)
- [4. Prompt Chain](#4-prompt-chain)
- [5. Chain Builder](#5-chain-builder)
- [6. Graph 기반 워크플로우](#6-graph-기반-워크플로우)
- [7. 실전 예제](#7-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ Chain을 사용하여 작업을 연결할 수 있습니다
- ✅ 순차적 및 병렬 실행을 구성할 수 있습니다
- ✅ 프롬프트 템플릿을 활용할 수 있습니다
- ✅ Graph 기반 워크플로우를 구성할 수 있습니다
- ✅ 복잡한 작업 파이프라인을 구축할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료
- [05_agent_and_tools.ipynb](05_agent_and_tools.ipynb) 완료 (권장)


## 1. Chain 기본 사용

Chain은 여러 LLM 호출을 연결하여 복잡한 작업을 수행합니다.

> 💡 **Chain의 핵심**:
> - 작업을 단계별로 나누어 처리
> - 이전 단계의 출력을 다음 단계의 입력으로 사용
> - 재사용 가능한 워크플로우 구성


In [ ]:
import asyncio
from beanllm import Client, Chain


async def basic_chain():
    """기본 Chain 예제"""

    # Client 생성
    client = Client(model="gpt-4o")

    # Chain 생성
    chain = Chain(client)

    # Chain 실행
    result = await chain.run("Python이란 무엇인가요? 간단히 설명해주세요.")

    print(f"✅ Chain 실행 완료!")
    print(f"📝 출력: {result.output}")
    print(f"\n📊 실행 단계: {len(result.steps)}개")
    for i, step in enumerate(result.steps, 1):
        print(f"  {i}. {step.get('type', 'unknown')}: {step.get('content', '')[:50]}...")

    return result


# 실행 (주석 해제)
# result = await basic_chain()
print("💡 위의 주석을 해제하여 실행하세요.")

## 2. Sequential Chain

여러 Chain을 순차적으로 실행합니다. 이전 Chain의 출력이 다음 Chain의 입력이 됩니다.


In [ ]:
from beanllm import SequentialChain, Chain, Client


async def sequential_chain_example():
    """Sequential Chain 예제"""

    client = Client(model="gpt-4o")

    # 여러 Chain 생성
    chain1 = Chain(client)  # 첫 번째 작업
    chain2 = Chain(client)  # 두 번째 작업
    chain3 = Chain(client)  # 세 번째 작업

    # Sequential Chain 생성
    sequential = SequentialChain([chain1, chain2, chain3])

    # 실행
    result = await sequential.run(input="Python 프로그래밍 언어에 대해 설명해주세요.")

    print(f"✅ Sequential Chain 실행 완료!")
    print(f"📝 최종 출력: {result.output[:200]}...")
    print(f"\n📊 총 단계: {len(result.steps)}개")

    return result


# 실행 (주석 해제)
# await sequential_chain_example()
print("💡 위의 주석을 해제하여 실행하세요.")

### 2.1 실제 사용 사례

텍스트 요약 → 번역 → 포맷팅 같은 실제 워크플로우를 구성해봅시다.


In [ ]:
async def text_processing_pipeline():
    """텍스트 처리 파이프라인"""

    client = Client(model="gpt-4o")

    # 1단계: 요약
    summary_chain = Chain(client)

    # 2단계: 번역 (요약 결과를 번역)
    translate_chain = Chain(client)

    # 3단계: 포맷팅 (번역 결과를 포맷팅)
    format_chain = Chain(client)

    # Sequential Chain 생성
    pipeline = SequentialChain([summary_chain, translate_chain, format_chain])

    long_text = """
    Python은 1991년에 발표된 고급 프로그래밍 언어입니다.
    간결하고 읽기 쉬운 문법이 특징입니다.
    동적 타입 시스템과 해석형 언어입니다.
    다양한 프로그래밍 패러다임을 지원합니다.
    """

    result = await pipeline.run(input=long_text)

    print("📝 원본 텍스트:")
    print(long_text[:100] + "...")
    print(f"\n✅ 처리 완료!")
    print(f"📄 최종 결과: {result.output[:200]}...")


# 실행 (주석 해제)
# await text_processing_pipeline()
print("💡 위의 주석을 해제하여 실행하세요.")

## 3. Parallel Chain

여러 Chain을 병렬로 실행하여 성능을 향상시킬 수 있습니다.


In [ ]:
from beanllm import ParallelChain, Chain, Client


async def parallel_chain_example():
    """Parallel Chain 예제"""

    client = Client(model="gpt-4o")

    # 여러 Chain 생성 (병렬 실행)
    chain1 = Chain(client)  # 작업 1
    chain2 = Chain(client)  # 작업 2
    chain3 = Chain(client)  # 작업 3

    # Parallel Chain 생성
    parallel = ParallelChain([chain1, chain2, chain3])

    # 실행 (모든 Chain이 동시에 실행됨)
    result = await parallel.run(input="Python의 특징을 설명해주세요.")

    print(f"✅ Parallel Chain 실행 완료!")
    print(f"📊 병렬 실행된 Chain 수: {len(result.steps)}개")
    print(f"📝 결과:")
    for i, step in enumerate(result.steps, 1):
        print(f"  {i}. {step.get('output', '')[:100]}...")

    return result


# 실행 (주석 해제)
# await parallel_chain_example()
print("💡 위의 주석을 해제하여 실행하세요.")

## 4. Prompt Chain

프롬프트 템플릿을 사용하여 일관된 형식의 출력을 생성할 수 있습니다.


In [ ]:
from beanllm import PromptChain, Client


async def prompt_chain_example():
    """Prompt Chain 예제"""

    client = Client(model="gpt-4o")

    # 프롬프트 템플릿
    template = """
    다음 텍스트를 분석하고 요약해주세요.
    
    텍스트: {input}
    
    요약:
    1. 주요 내용
    2. 핵심 포인트
    3. 결론
    """

    # Prompt Chain 생성
    prompt_chain = PromptChain(client, template)

    # 실행
    text = "Python은 간결하고 읽기 쉬운 문법이 특징입니다."
    result = await prompt_chain.run(input=text)

    print(f"✅ Prompt Chain 실행 완료!")
    print(f"📝 결과:\n{result.output}")


# 실행 (주석 해제)
# await prompt_chain_example()
print("💡 위의 주석을 해제하여 실행하세요.")

## 5. Chain Builder

Fluent API를 사용하여 Chain을 쉽게 구성할 수 있습니다.


In [ ]:
from beanllm import ChainBuilder, Client


async def chain_builder_example():
    """Chain Builder 예제"""

    client = Client(model="gpt-4o")

    # Fluent API로 Chain 구성
    chain = ChainBuilder(client).with_template("다음 텍스트를 요약: {input}").verbose(True).build()

    result = await chain.run(input="Python은 프로그래밍 언어입니다.")

    print(f"✅ Chain Builder로 생성된 Chain 실행 완료!")
    print(f"📝 결과: {result.output}")


# 실행 (주석 해제)
# await chain_builder_example()
print("💡 위의 주석을 해제하여 실행하세요.")

## 6. Graph 기반 워크플로우

복잡한 조건부 분기와 루프를 포함한 워크플로우를 구성할 수 있습니다.

### 6.1 기본 Graph

노드와 엣지로 구성된 그래프 워크플로우입니다.


In [ ]:
from beanllm import Graph, Client


async def graph_example():
    """Graph 예제"""

    client = Client(model="gpt-4o")

    # Graph 생성
    graph = Graph(client)

    # 노드 추가
    graph.add_node("start", lambda x: f"시작: {x}")
    graph.add_node("process", lambda x: f"처리: {x.upper()}")
    graph.add_node("end", lambda x: f"완료: {x}")

    # 엣지 추가 (노드 간 연결)
    graph.add_edge("start", "process")
    graph.add_edge("process", "end")

    # 실행
    result = await graph.run("테스트")

    print(f"✅ Graph 실행 완료!")
    print(f"📝 결과: {result}")

    return result


# 실행 (주석 해제)
# await graph_example()
print("💡 위의 주석을 해제하여 실행하세요.")

### 6.2 조건부 분기

조건에 따라 다른 경로로 실행할 수 있습니다.


In [ ]:
async def conditional_graph():
    """조건부 분기 Graph"""

    client = Client(model="gpt-4o")

    graph = Graph(client)

    # 노드 추가
    graph.add_node("input", lambda x: x)
    graph.add_node("check", lambda x: len(x) > 10)  # 조건 체크
    graph.add_node("short", lambda x: f"짧은 텍스트: {x}")
    graph.add_node("long", lambda x: f"긴 텍스트: {x}")

    # 엣지 추가
    graph.add_edge("input", "check")
    graph.add_conditional_edge("check", "short", lambda x: not x)  # False면 short
    graph.add_conditional_edge("check", "long", lambda x: x)  # True면 long

    # 실행
    result1 = await graph.run("짧음")
    result2 = await graph.run("이것은 매우 긴 텍스트입니다")

    print(f"✅ 짧은 텍스트 결과: {result1}")
    print(f"✅ 긴 텍스트 결과: {result2}")


# 실행 (주석 해제)
# await conditional_graph()
print("💡 위의 주석을 해제하여 실행하세요.")

## 7. 실전 예제

### 7.1 문서 처리 파이프라인

여러 단계를 거쳐 문서를 처리하는 파이프라인입니다.


In [ ]:
async def document_processing_pipeline():
    """문서 처리 파이프라인"""

    client = Client(model="gpt-4o")

    # 1. 문서 로딩 Chain
    load_chain = Chain(client)

    # 2. 요약 Chain
    summary_chain = PromptChain(client, "다음 문서를 요약해주세요:\n{input}\n\n요약:")

    # 3. 키워드 추출 Chain
    keyword_chain = PromptChain(
        client, "다음 텍스트에서 주요 키워드를 추출해주세요:\n{input}\n\n키워드:"
    )

    # Sequential Chain으로 연결
    pipeline = SequentialChain([load_chain, summary_chain, keyword_chain])

    document = "Python은 프로그래밍 언어입니다. 간결하고 읽기 쉬운 문법이 특징입니다."

    result = await pipeline.run(input=document)

    print("📄 문서 처리 파이프라인 실행 완료!")
    print(f"📝 최종 결과: {result.output}")


# 실행 (주석 해제)
# await document_processing_pipeline()
print("💡 위의 주석을 해제하여 실행하세요.")

### 7.2 병렬 분석 시스템

여러 관점에서 동시에 분석하는 시스템입니다.


In [ ]:
async def parallel_analysis():
    """병렬 분석 시스템"""

    client = Client(model="gpt-4o")

    # 여러 분석 Chain 생성
    sentiment_chain = PromptChain(client, "다음 텍스트의 감정을 분석해주세요: {input}")

    topic_chain = PromptChain(client, "다음 텍스트의 주제를 분석해주세요: {input}")

    style_chain = PromptChain(client, "다음 텍스트의 스타일을 분석해주세요: {input}")

    # Parallel Chain으로 연결
    analysis = ParallelChain([sentiment_chain, topic_chain, style_chain])

    text = "Python은 정말 훌륭한 프로그래밍 언어입니다!"

    result = await analysis.run(input=text)

    print("📊 병렬 분석 완료!")
    print(f"📝 결과: {len(result.steps)}개 분석 완료")


# 실행 (주석 해제)
# await parallel_analysis()
print("💡 위의 주석을 해제하여 실행하세요.")

## 📸 시각화 예제

### Chain 구조 시각화

> 💡 **이미지 필요**: Chain의 구조를 보여주는 다이어그램

```python
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(12, 6))

# Sequential Chain 구조
nodes = ["Input", "Chain 1", "Chain 2", "Chain 3", "Output"]
x_positions = [0, 2, 4, 6, 8]

for i, (node, x) in enumerate(zip(nodes, x_positions)):
    color = "lightblue" if i == 0 or i == len(nodes)-1 else "lightgreen"
    rect = mpatches.Rectangle((x-0.5, -0.3), 1, 0.6, facecolor=color, edgecolor='black')
    ax.add_patch(rect)
    ax.text(x, 0, node, ha='center', va='center', fontsize=10)
    
    if i < len(nodes) - 1:
        ax.arrow(x+0.5, 0, 1, 0, head_width=0.1, head_length=0.1, fc='black', ec='black')

ax.set_xlim(-1, 9)
ax.set_ylim(-1, 1)
ax.axis('off')
plt.title('Sequential Chain 구조', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
```

**필요한 이미지**:
- 플로우차트: Sequential Chain의 선형 구조
- Input → Chain 1 → Chain 2 → Chain 3 → Output
- 각 노드는 박스로 표시
- 화살표로 데이터 흐름 표시
- 병렬 Chain의 경우 여러 경로가 동시에 실행되는 구조


## 8. 다음 단계

Chain과 Graph를 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[07_multi_agent.ipynb](07_multi_agent.ipynb)** - Multi-Agent
   - 여러 Agent 협업
   - 에이전트 간 통신

2. **[08_vision_rag.ipynb](08_vision_rag.ipynb)** - Vision RAG
   - 이미지 기반 RAG
   - 멀티모달 검색

3. **[09_ocr.ipynb](09_ocr.ipynb)** - OCR
   - 이미지에서 텍스트 추출
   - 문서 디지털화

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#chain) - Chain API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ Chain & Graph 완료! 다음 노트북으로 진행하세요.**
ㄱㄱ